<a href="https://colab.research.google.com/github/wakepon21/Masa/blob/master/signate1_25.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dataset

## import

In [6]:
# Dataset
from sklearn.datasets import make_classification
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import train_test_split

# LightGBM
import lightgbm as lgb
from sklearn.metrics import roc_auc_score

# LogisticRegression
from sklearn.linear_model import LogisticRegression

# Imbalanced-learn
from imblearn.under_sampling import RandomUnderSampler


In [7]:
#control
#!pip install git+https://github.com/pandas-profiling/pandas-profiling.git
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
#import pandas_profiling
import seaborn as sns
#from pandas_profiling.utils.cache import cache_file

#Optuna
!pip install optuna
import optuna
from sklearn.metrics import log_loss

#何で必要なのか分かってない
%matplotlib inline
warnings.filterwarnings('ignore')

     |████████████████████████████████| 235kB 2.7MB/s 
     |████████████████████████████████| 1.1MB 11.2MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 81kB 6.5MB/s 
     |████████████████████████████████| 81kB 6.5MB/s 
     |████████████████████████████████| 133kB 23.1MB/s 
     |████████████████████████████████| 51kB 6.5MB/s 
     |████████████████████████████████| 112kB 18.6MB/s 
  Created wheel for alembic: filename=alembic-1.4.2-cp36-none-any.whl size=159543 sha256=f9183af71576abc66b757471fb539a3141f4b1dc258d95b61d965c2d3aef7285
  Stored in directory: /root/.cache/pip/wheels/1f/04/83/76023f7a4c14688c0b5c2682a96392cfdd3ee4449eaaa287ef
Successfully built alembic
  Created wheel for optuna: filename=optuna-2.0.0-cp36-none-any.whl size=312964 sha256=4583c126d063137703e95f1fb251eadb349e5fa3a8b65b9ab1dd93c0406efe58
  Stored in directory: /root/.cache/pip/wheels/b5

## upload

In [8]:
#train,test,submit_sampleのみっつがそろっているか確認
!ls

sample_data


In [9]:
#そろっていなかったら選択
from google.colab import files
train_up = files.upload()

Saving submit_sample.csv to submit_sample.csv
Saving test.csv to test.csv
Saving train.csv to train.csv


## Feature Engineering

In [10]:
sub = pd.read_csv("submit_sample.csv",names=("A","B"))
test = pd.read_csv("test.csv")
train = pd.read_csv("train.csv")
data = pd.concat([train,test], sort=False)

#特徴量をエンジニアリング
data["income"]=data["job"].copy()
data["goodincome"]=data["job"].copy()
data["job"].replace(['blue-collar','management','technician','admin.','services',\
                     'unknown','self-employed','entrepreneur','student','retired',\
                     'unemployed','housemaid'],[0,1,2,3,4,5,6,7,8,9,10,11],inplace=True)
data["loan"].replace(['yes','no'],[1,0],inplace=True)
data["married"]=data["marital"]
data["single"]=data["marital"]
data['education'].replace(['secondary','tertiary','primary','unknown'],[3,1,0,2],inplace=True)
data["housing"].replace(['yes','no'],[1,0],inplace=True)
data["contact"].replace(['cellular','telephone','unknown'],[2,1,0],inplace=True)
data["poutcome"].replace(['success','failure','unknown','other'],[3,1,2,0],inplace=True)
#data["job"] = data["job"].astype(np.int64)

data["month"].replace(['jan','feb','mar','apr','may','jun','jul','aug','sep','oct','nov','des'],\
                      [3,4,5,6,7,8,9,10,11,0,1,2],inplace=True)
#data["y"].astype(np.int64)
#ここから追加
data["year"]=data["month"]*30+data["day"]
data["campaign+previous"]=data["campaign"] + data["previous"]
data["poutcome/pdays"]=data["poutcome"] / (data["pdays"]+10) * 10000
data["poutcome/pdays"] = data["poutcome/pdays"].astype(np.int64)
data["income"].replace(['blue-collar','management','technician',\
                                         'admin.','services','unknown','self-employed',\
                                         'entrepreneur','student','retired','unemployed','housemaid'],\
                                        [0,0,0,0,0,0,0,0,1,1,1,0],inplace=True)
data["goodincome"].replace(['blue-collar','management','technician','admin.','services','unknown','self-employed','entrepreneur','student','retired','unemployed','housemaid'],\
                                                [0,1,0,1,0,0,1,1,0,0,0,0],inplace=True)
data["housing+loan"] = data["housing"] + data["loan"]
data["duration*campaign"] = data["duration"] * data["campaign"] 
data["married"].replace(['married','single','divorced'],[1,0,1],inplace=True)
data["single"].replace(['married','single','divorced'],[1,0,0],inplace=True)
train["y"]=train["y"].astype(np.int64)

#全体のうち、残したいモノだけ選ぶ
all_columns = list(train.columns)
remain_columns = ['age','job','education','balance','housing','loan',
                  'contact','day','month','duration','campaign','pdays','previous',
                  'poutcome','y','income','goodincome',"married","single","year",
                  'campaign+previous'	,'poutcome/pdays'	,'housing+loan','duration*campaign']
                  
delete_columns = list(set(all_columns)-set(remain_columns))
data.drop(delete_columns, axis=1, inplace=True)

categorical_features = ['job','education','housing','loan','contact','poutcome',
                        "income",'goodincome','married',"single",'housing+loan']
data[categorical_features] = data[categorical_features].astype('category')

#trainとtestを再度切り分け
train = data[:len(train)]
test = data[len(train):]

#train,testを、さらに説明変数Xと、予測変数yに切り分け。y_testは与えられていないのでなし。三種類がでる
y = train['y']
X = train.drop('y', axis = 1)
X_sub = test.drop('y', axis = 1)

Xcolumns=X.columns
#ycolumns=y.columns



# LightGBM

## def LGB

In [23]:
#LightGBMのデータ分割
X_train_LGB, X_test_LGB, y_train_LGB, y_test_LGB = train_test_split(X, y, test_size=0.2, random_state=0)
X_train2_LGB, X_valid_LGB, y_train2_LGB, y_valid_LGB = train_test_split(X, y, test_size=0.2, random_state=1)

In [24]:
def lgbm_train(X_train_df, X_valid_df, y_train_df, y_valid_df, lgbm_params):
    lgb_train = lgb.Dataset(X_train_df, y_train_df)
    lgb_eval = lgb.Dataset(X_valid_df, y_valid_df, reference=lgb_train)

    # 上記のパラメータでモデルを学習する
    model = lgb.train(lgbm_params, lgb_train,
                      # モデルの評価用データを渡す
                      valid_sets=lgb_eval,
                      # 最大で 1000 ラウンドまで学習する
                      num_boost_round=1000,
                      # 10 ラウンド経過しても性能が向上しないときは学習を打ち切る
                      early_stopping_rounds=10)
    
    return model

## Optuna of LGB

In [25]:
def objective(trial):
    params = {
        'objective': 'binary',
        'max_bin': trial.suggest_int('max_bin', 255, 500),
        'learning_rate': 0.15,
        'num_leaves': trial.suggest_int('num_leaves', 32, 128),
    }

    lgb_train = lgb.Dataset(X_train_LGB, y_train_LGB)
    lgb_eval = lgb.Dataset(X_valid_LGB, y_valid_LGB, reference=lgb_train)

    model = lgb.train(params, lgb_train,
                                   valid_sets=[lgb_train, lgb_eval],
                                   verbose_eval=10,
                                   num_boost_round=1000,
                                   early_stopping_rounds=10)

    y_pred_valid = model.predict(X_valid, num_iteration=model.best_iteration)
    score = log_loss(y_valid, y_pred_valid)
    return score

In [26]:
study = optuna.create_study(sampler=optuna.samplers.RandomSampler(seed=0))
study.optimize(objective, n_trials=2)

Training until validation scores don't improve for 10 rounds.
[10]	training's binary_logloss: 0.183185	valid_1's binary_logloss: 0.165843
[20]	training's binary_logloss: 0.152312	valid_1's binary_logloss: 0.141168
[30]	training's binary_logloss: 0.130338	valid_1's binary_logloss: 0.126298
[40]	training's binary_logloss: 0.113954	valid_1's binary_logloss: 0.113959
[50]	training's binary_logloss: 0.100857	valid_1's binary_logloss: 0.105269
[60]	training's binary_logloss: 0.0905326	valid_1's binary_logloss: 0.0984232
[70]	training's binary_logloss: 0.0813099	valid_1's binary_logloss: 0.0919385
[80]	training's binary_logloss: 0.0734049	valid_1's binary_logloss: 0.0864496
[90]	training's binary_logloss: 0.0663634	valid_1's binary_logloss: 0.0821787
[100]	training's binary_logloss: 0.0603149	valid_1's binary_logloss: 0.0781767
[110]	training's binary_logloss: 0.0541766	valid_1's binary_logloss: 0.0743709
[120]	training's binary_logloss: 0.0486519	valid_1's binary_logloss: 0.0700578
[130]	tra

[I 2020-08-23 12:37:03,115] Trial 0 finished with value: 0.048523823989085675 and parameters: {'max_bin': 427, 'num_leaves': 79}. Best is trial 0 with value: 0.048523823989085675.


Training until validation scores don't improve for 10 rounds.
[10]	training's binary_logloss: 0.176956	valid_1's binary_logloss: 0.16222
[20]	training's binary_logloss: 0.144301	valid_1's binary_logloss: 0.137977
[30]	training's binary_logloss: 0.119363	valid_1's binary_logloss: 0.120048
[40]	training's binary_logloss: 0.101942	valid_1's binary_logloss: 0.107525
[50]	training's binary_logloss: 0.0882003	valid_1's binary_logloss: 0.0982029
[60]	training's binary_logloss: 0.0775942	valid_1's binary_logloss: 0.090586
[70]	training's binary_logloss: 0.0682231	valid_1's binary_logloss: 0.0845657
[80]	training's binary_logloss: 0.0605564	valid_1's binary_logloss: 0.0796208
[90]	training's binary_logloss: 0.0532918	valid_1's binary_logloss: 0.0741255
[100]	training's binary_logloss: 0.0481037	valid_1's binary_logloss: 0.0706625
[110]	training's binary_logloss: 0.0429041	valid_1's binary_logloss: 0.0670822
[120]	training's binary_logloss: 0.0385013	valid_1's binary_logloss: 0.0642672
[130]	tra

[I 2020-08-23 12:37:05,953] Trial 1 finished with value: 0.04910344546188399 and parameters: {'max_bin': 372, 'num_leaves': 96}. Best is trial 0 with value: 0.048523823989085675.


In [27]:
lgbm_params = {
    'learning_rate': 0.05,
    'num_leaves': study.best_params["num_leaves"],
    'boosting_type' : 'gbdt',
    'objective': 'binary',
    'max_bin': study.best_params["max_bin"],
    'max_dapth': 7
}



##Bagging of LGB

In [32]:
def bagging(seed):
    positive_count_train = y_train_LGB.sum()
    sampler = RandomUnderSampler(sampling_strategy={0:int(positive_count_train), 1:int(positive_count_train)},random_state=seed, replacement=True)
    X_resampled, y_resampled = sampler.fit_resample(X_train_LGB, y_train_LGB)
    X_train2_LGB, X_valid_LGB, y_train2_LGB, y_valid_LGB = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=seed)
    model_bagging = lgbm_train(X_train2_LGB, X_valid_LGB, y_train2_LGB, y_valid_LGB, lgbm_params)
    return model_bagging

In [33]:
%%time
models_LGB = []

for i in range(10):
    models_LGB.append(bagging(i))

[1]	valid_0's binary_logloss: 0.677179
Training until validation scores don't improve for 10 rounds.
[2]	valid_0's binary_logloss: 0.66108
[3]	valid_0's binary_logloss: 0.645978
[4]	valid_0's binary_logloss: 0.633388
[5]	valid_0's binary_logloss: 0.620767
[6]	valid_0's binary_logloss: 0.609628
[7]	valid_0's binary_logloss: 0.598957
[8]	valid_0's binary_logloss: 0.588639
[9]	valid_0's binary_logloss: 0.579988
[10]	valid_0's binary_logloss: 0.571694
[11]	valid_0's binary_logloss: 0.563792
[12]	valid_0's binary_logloss: 0.557077
[13]	valid_0's binary_logloss: 0.551651
[14]	valid_0's binary_logloss: 0.544969
[15]	valid_0's binary_logloss: 0.537791
[16]	valid_0's binary_logloss: 0.532016
[17]	valid_0's binary_logloss: 0.526657
[18]	valid_0's binary_logloss: 0.52232
[19]	valid_0's binary_logloss: 0.517635
[20]	valid_0's binary_logloss: 0.512757
[21]	valid_0's binary_logloss: 0.506361
[22]	valid_0's binary_logloss: 0.502166
[23]	valid_0's binary_logloss: 0.498177
[24]	valid_0's binary_logloss

In [74]:
y_preds_LGB = []

for m in models_LGB:
    y_preds_LGB.append(m.predict(X_test_LGB, num_iteration=m.best_iteration))

y_preds_bagging_LGB = sum(y_preds_LGB)/len(y_preds_LGB)
# auc を計算する
auc = roc_auc_score(y_test_LGB, y_preds_bagging_LGB)
print(auc)

for y_pred_LGB in y_preds_LGB:
    print(roc_auc_score(y_test_LGB, y_pred_LGB))

0.8655819895845333
0.8276603701535495
0.8306342641611058
0.8375461745171894
0.83403683231372
0.8303481307440364
0.8363790236854299
0.8430542427957759
0.8326161588587484
0.8316767576034432
0.851518321743352


In [75]:
y_subs_LGB = []

for m in models_LGB:
    y_subs_LGB.append(m.predict(X_sub, num_iteration=m.best_iteration))

y_subs_bagging_LGB = sum(y_subs_LGB)/len(y_subs_LGB)
# auc を計算する
y_subs_bagging_LGB

array([0.9668189 , 0.33883675, 0.08978856, ..., 0.2196017 , 0.01573662,
       0.56981533])

In [76]:
percentage_LGB=0.18
y_sub_LGB = (y_subs_bagging_LGB > percentage_LGB).astype(int)

sub['B'] = y_sub_LGB
sub.to_csv('submission_lightgbm_bagging.csv', index=False, header=False)

sum(sub["B"]),len(sub["B"])

(10219, 18050)

# LogisticRegression


## def LogiReg

In [77]:
#LogisticRegressionのデータ分割
X_train_LR, X_test_LR, y_train_LR, y_test_LR = train_test_split(X, y, test_size=0.2, random_state=11)
#X_train2, X_valid, y_train2, y_valid = train_test_split(X, y, test_size=0.2, random_state=1)

In [78]:
def LogiReg(seed):
    positive_count_train = y_train_LR.sum()
    sampler_LR = RandomUnderSampler(sampling_strategy={0:int(positive_count_train), 1:int(positive_count_train)},random_state=seed, replacement=True)
    X_resampled_LR, y_resampled_LR = sampler_LR.fit_resample(X_train_LR, y_train_LR)
    clf = LogisticRegression(penalty='l2', solver="sag", random_state=seed)
    clf.fit(X_resampled_LR, y_resampled_LR)

    return clf

## Bagging of LogiReg

In [79]:
%%time
models_LR = []

for i in range(10):
    models_LR.append(LogiReg(i))

CPU times: user 1.21 s, sys: 9.96 ms, total: 1.22 s
Wall time: 1.22 s


In [87]:
y_preds_LR = []
percentage_LR=0.2

for clf in models_LR:
    y_preds_LR.append(clf.predict(X_sub))
y_subs_bagging_LR = sum(y_preds_LR)/len(y_preds_LR)
y_sub_LR = (y_subs_bagging_LR > percentage_LR).astype(int)

sub['B'] = y_sub_LR
sub.to_csv('submission_LogisticRegression_bagging.csv', index=False, header=False)

sum(sub["B"]),len(sub["B"])

(9358, 18050)

# ensemble

# Output

In [ ]:
files.download('submission_lightgbm_kfold.csv')